In [0]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.0.0-dev20190413


In [0]:
# Download and unzip the complete dogs-vs-cats dataset

%env DIR=cats_and_dogs

! curl -o dogs-vs-cats.zip https://storage.googleapis.com/peddy-ai-dl-data/deeplearning-repo/dogs-vs-cats.zip

! unzip -o dogs-vs-cats.zip -d $DIR
! mkdir $DIR/original_data
! unzip -o $DIR/test1.zip -d $DIR/original_data
! unzip -o $DIR/train.zip -d $DIR/original_data

In [0]:
# Create train/validation/test datasets from original data
import os, shutil

USE_DATA_AUGMENTATION = True

base_dir = os.path.join(os.getcwd(), 'cats_and_dogs')
original_dataset_dir = os.path.join(base_dir, 'original_data', 'train')

data_segment_names = ['train', 'validation', 'test']
labels = ['cat', 'dog']

data_segments = {
	data_segment_names[0]: (0, 1000),
	data_segment_names[1]: (1000, 1500),
	data_segment_names[2]: (1500, 2000)
}

def mkdir(dir):
	try:
		os.mkdir(dir)
	except OSError:
		print('{} not created - alread exists'.format(dir), dir)

def prepare_data():
	# Make data segments directories
	for segment in data_segment_names:
		segment_dir = os.path.join(base_dir, segment)
		mkdir(segment_dir)
		# Make label directories for each data segment
		for label in labels:
			label_segment_dir = os.path.join(segment_dir, label + 's')
			mkdir(label_segment_dir)
			# Copy data over for each data segment-label pair
			fnames = [(label+'.{}.jpg').format(i) for i in range(data_segments[segment][0], data_segments[segment][1])]
			for fname in fnames:
				src = os.path.join(original_dataset_dir, fname)
				dst = os.path.join(label_segment_dir, fname)
				shutil.copyfile(src, dst)

prepare_data()

In [0]:
# Import VGG16 base, set up dir structure and init DataGen
conv_base = VGG16(weights='imagenet',
				  include_top=False,
				  input_shape=(150, 150, 3))

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 10

In [0]:
# Returns VGG16 feature vectores and labels for sample_count images in directory
def extract_features(directory, sample_count):
	features = np.zeros(shape=(sample_count, 4, 4, 512))
	labels = np.zeros(shape=(sample_count))
	generator = datagen.flow_from_directory(
		directory,
		target_size=(150, 150),
		batch_size=batch_size,
		class_mode='binary')
	i = 0
	for input_batch, labels_batch in generator:
		features_batch = conv_base.predict(input_batch)
		features[i * batch_size : (i+1) * batch_size] = features_batch
		labels[i * batch_size : (i+1) * batch_size] = labels_batch
		i += 1
		if i * batch_size >= sample_count:
			break
	return features, labels

In [0]:
# Obtain and flatten feature+labels for train/validation/test
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
# Build and compile the model with feature vectors as input
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
	loss='binary_crossentropy',
	metrics=['acc'])


In [0]:
# Train and evaluate the model
history = model.fit(train_features, train_labels,
	epochs=30,
	batch_size=20,
	validation_data=(validation_features, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.5741 - acc: 0.6875 - val_loss: 0.4269 - val_acc: 0.8300
Epoch 2/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.4134 - acc: 0.8260 - val_loss: 0.3522 - val_acc: 0.8730
Epoch 3/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.3529 - acc: 0.8475 - val_loss: 0.3150 - val_acc: 0.8830
Epoch 4/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.3145 - acc: 0.8670 - val_loss: 0.2931 - val_acc: 0.8870
Epoch 5/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.2886 - acc: 0.8840 - val_loss: 0.2797 - val_acc: 0.8900
Epoch 6/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.2603 - acc: 0.8930 - val_loss: 0.2768 - val_acc: 0.8930
Epoch 7/30
2000/2000 [==============================] - 3s 1ms/sample - loss: 0.2439 - acc: 0.9065 - val_loss: 0.2730 - val_acc: 0.8950
